In [3]:
import functools
import operator

In [1]:
import h5py
import numpy as np

In [2]:
import lib

In [4]:
def get_weight(weight_dataset, weight_keys=["statistical_weight"]):
    return functools.reduce(
        operator.mul,
        [
            weight_dataset[weight_key][:]
            for weight_key in weight_keys
        ],
    )     

In [5]:
shear_step_plus = "g1_slice=0.02__g2_slice=0.00__g1_other=0.00__g2_other=0.00__zlow=0.0__zhigh=6.0"
shear_step_minus = "g1_slice=-0.02__g2_slice=0.00__g1_other=0.00__g2_other=0.00__zlow=0.0__zhigh=6.0"

In [6]:
weight_keys = ["statistical_weight"]

In [18]:
for shear_step in [shear_step_plus, shear_step_minus]:
    print(shear_step)
    print()
    with (
        h5py.File(lib.const.SIM_SHEAR_CATALOGS[shear_step]) as shear,
        # h5py.File(lib.const.SIM_MATCH_CATALOGS[shear_step]) as truth,
        h5py.File(lib.const.SIM_TOMOGRAPHY_CATALOGS[shear_step]) as tomo,
        h5py.File(lib.const.SIM_WEIGHT_CATALOGS[shear_step]) as weight,
    ):
        bhat = tomo["sompz"]["noshear"]["bhat"][:]
    
        for weight_keys in [
            ["statistical_weight"],
            ["statistical_weight", "neighbor_weight"],
            ["statistical_weight", "occupancy_weight"],
            ["statistical_weight", "neighbor_weight", "occupancy_weight"],
        ]:
            print(weight_keys)
            print("b", "\t", "w", "\t\t\t", "w * R")
            w = get_weight(weight["mdet"]["noshear"], weight_keys=weight_keys)
            r = lib.response.get_shear_response(shear["mdet"]["noshear"])
            for tomographic_bin in lib.const.TOMOGRAPHIC_BINS:
                in_tomo = (bhat == tomographic_bin)
                print(
                    tomographic_bin,
                    "\t",
                    np.sum(w[in_tomo]),
                    "\t",
                    np.sum((w * r)[in_tomo]),
                )
            print()
            

g1_slice=0.02__g2_slice=0.00__g1_other=0.00__g2_other=0.00__zlow=0.0__zhigh=6.0

['statistical_weight']
b 	 w 			 w * R
0 	 37687408.52750142 	 31852021.778152555
1 	 32262590.501660284 	 25166769.445796162
2 	 36321284.070149 	 28019787.27996662
3 	 22979366.319672316 	 14517510.901062118

['statistical_weight', 'neighbor_weight']
b 	 w 			 w * R
0 	 36935272.88896909 	 31258164.71036748
1 	 33630265.55123577 	 26647800.06951995
2 	 37616957.073409796 	 29500162.15844555
3 	 23442093.917962722 	 15133209.390449103

['statistical_weight', 'occupancy_weight']
b 	 w 			 w * R
0 	 38270056.368683755 	 32061262.610645365
1 	 35022342.94013908 	 27517630.555414688
2 	 30616116.814313516 	 23831877.593959697
3 	 25344050.202425964 	 15939348.02708475

['statistical_weight', 'neighbor_weight', 'occupancy_weight']
b 	 w 			 w * R
0 	 38219183.82005287 	 32044282.53917939
1 	 36907118.418624476 	 29395545.456652362
2 	 32107396.97912129 	 25375065.53263439
3 	 25919483.487100307 	 16645228.6516